In [1]:
#---------------------------
# V2

In [2]:
# importi
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [3]:
stStavkov = 100
potDoModela = "modeli\\T5_Clarin_50"
potDoModela = "modeli\\T5_Clarin_100"
potDoModela = "modeli\\T5_Clarin_150"
potDoModela = "modeli\\T5_Clarin_200"
potDoModela = "modeli\\T5_Google_50"
potDoModela = "modeli\\T5_Google_100"
potDoModela = "modeli\\T5_Google_150"
potDoModela = "modeli\\T5_Google_200"
potDoModela = "modeli\\T5_Deepl_50"
potDoModela = "modeli\\T5_Deepl_100"
potDoModela = "modeli\\T5_Deepl_150"
potDoModela = "modeli\\T5_Deepl_200"
stParafraz = 10 

In [4]:
testniStavki = pd.read_excel("TestniStavki_ZaTest.xlsx")
testniStavki = testniStavki.values.tolist()
testniStavki = [i[0] for i in testniStavki]
testniStavki = testniStavki[:stStavkov] #Zadokumentirat če bi celotno bazo
print(len(testniStavki))

100


In [5]:
tokenizer = T5Tokenizer.from_pretrained("cjvt/t5-sl-small")
model = T5ForConditionalGeneration.from_pretrained(potDoModela)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [6]:
from datetime import datetime
zacetek = datetime.now()

celotnaTabela2 = pd.DataFrame(index=range(len(testniStavki)),columns=range(stParafraz+1))


for i in range(len(testniStavki)):
    text =  "parafraza: " + testniStavki[i]
    max_len = 256
    vrstica = [testniStavki[i]]
    celotnaTabela2[0][i] = testniStavki[i]
    #encoding = tokenizer.encode_plus(text, pad_to_max_length=True, return_tensors="pt")
    encoding = tokenizer.encode_plus(text, padding='max_length', return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    
    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        #num_return_sequences=10
        num_return_sequences=stParafraz
    )
    

    print ("\nOriginalni stavek:")
    print (testniStavki[i])
    print ("\n")
    print ("Parafrazirani stavki:")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != testniStavki[i].lower() and sent not in final_outputs:
            final_outputs.append(sent)
            
    for a, final_output in enumerate(final_outputs):
        print("{}: {}".format(a, final_output))
        celotnaTabela2[a+1][i] = final_output
        vrstica.append(final_output)
        
    print("\n-----------------------\n")
    
konec = datetime.now()
print(konec-zacetek)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.



Originalni stavek:
Vsaka občinska volilna komisija mora v občini določiti najmanj eno volišče, ki je dostopno invalidom.


Parafrazirani stavki:
0: Občinska volilna komisija mora v občini določiti vsaj eno volišče, ki je dostopno invalidom.
1: Vsaka občinska volilna komisija mora v občini določiti vsaj eno volišče, ki je dostopno invalidom.
2: Občinska volilna komisija mora za vsako občino določiti najmanj eno volišče v občini.
3: Občinska volilna komisija mora v občini zagotoviti vsaj eno volišče, ki je dostopno invalidom.
4: Občinska volilna komisija mora za vsako občino določiti vsaj eno volišče, ki je dostopno invalidom.

-----------------------


Originalni stavek:
Če nič ne delajo, ne morejo nič narediti narobe in jih nihče ne more pribijati na križ, pravi Simič.


Parafrazirani stavki:

-----------------------


Originalni stavek:
Po svoji naravi je svetovljanka in se znajde v katerikoli deželi, povsod je lahko doma.


Parafrazirani stavki:
0: Po svoji naravi je svetovljan in s

In [7]:
# Koda za shranjevanje v Excel
# shranim še tabelo prevedenih
skupna = pd.DataFrame(np.column_stack([celotnaTabela2]), 
                               columns=["Original","Parafraza 1","Parafraza 2","Parafraza 3","Parafraza 4","Parafraza 5","Parafraza 6","Parafraza 7","Parafraza 8","Parafraza 9","Parafraza 10"])

skupna.to_excel(potDoModela+".xlsx")